In [1]:
# 

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#Import CRUD Python module file name and class name
from PythonCRUD import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# Sign into MongoDB with username and password
username = "aacuser"
password = "mypass"
shelter = AnimalShelter(username, password)


#Default load of database values via cursor object
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Added in Grazioso Salvare’s Logo
image_filename = 'Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    #My unique identifier
    html.Center(html.B(html.H1('Written by: Nathan Anglin'))),
    html.Hr(),
    html.Br(),
    html.Br(),
    html.B('Select Rescue Animal Type:'),
    html.Br(),
    # Added dropdown box for interactive filtering options.
    html.Div(className='row',
             style={'flex-wrap': 'nowrap'},
             children=[dcc.Dropdown(id='dropdown-button', options=[
                 {"label": "All Animals", "value": "Reset"},
                 {"label": "Water", "value": 'Water'},
                 {"label": "Mountain / Wilderness", "value": "Mountain"},
                 {"label": "Disaster / Individual Tracking", "value": "Disaster"},
             ],
            value='Reset',
            clearable=False)]),
    html.Br(),
    html.Br(),
    html.Hr(),
    
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #Interactive Data Table Configuration
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        
    ),
    html.Br(),
    html.Hr(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m16',

            ),
        html.Div(
            id='map-id',
            className='col s12 m16',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('dropdown-button', 'value')])
def update_dashboard(dropdown_choice):
    #Load specific datasets matching the types of Dogs needed for training based on Dropown Menu.
    if dropdown_choice == "Water":
        df = pd.DataFrame.from_records(shelter.read({'$and': [
        {'$or': [ {'breed':'Labrador Retriever Mix'}, {'breed':'Chesapeake Bay Retriever'},
        {'breed':'Newfoundland'}]},
        {'sex_upon_outcome':'Intact Female'}, {'age_upon_outcome_in_weeks':{'$lte':156, '$gte':26}}]}))      
    elif dropdown_choice == "Mountain":
        df = pd.DataFrame.from_records(shelter.read({'$and': [
        {'$or': [ {'breed':'German Shepherd'}, {'breed':'Alaskan Malamute'},
        {'breed':'Old English Sheepdog'},{'breed':'Siberian Husky'},{'breed':'Rottweiler'}]},
        {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':156, '$gte':26}}]}))
    elif dropdown_choice == "Disaster":
        df = pd.DataFrame.from_records(shelter.read({'$and': [
        {'$or': [ {'breed':'Doberman Pinscher'}, {'breed':'German Sheperd'},
        {'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]},
        {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':300, '$gte':20}}]}))
    elif dropdown_choice == "Reset":
        df = pd.Dataframe.from_records(shelter.read({}))

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
               
    return (data,columns)


"""
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
"""
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    #Added code to display percentanges of each Breed available with current filter
    df = pd.DataFrame.from_dict(viewData)
    breed_dict = df["breed"].value_counts()
    df_pie = pd.DataFrame(breed_dict)
    df_pie = df_pie.reset_index()
    df_pie.columns = ['breed', 'total']
    fig = px.pie(df_pie, values='total', names='breed', hole=.3, title='Current Page: Available Breeds Matching Filter')
    
    return [
        dcc.Graph(figure = fig)               
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    #Added in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, 
        center=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app